In [4]:
# === Imports ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_scheduler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import ParameterGrid
from torch.optim import AdamW
from tqdm import tqdm
import pickle
import csv
import os

# === Load Data ===
train_df = pd.read_csv("splits/train.csv")
val_df = pd.read_csv("splits/val.csv")

# === Load Label Encoder ===
with open("splits/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

num_classes = len(label_encoder.classes_)

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# === Dataset ===
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=192):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encodings = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors="pt")
        return {
            "input_ids": encodings["input_ids"].squeeze(0),
            "attention_mask": encodings["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# === Model ===
class BERTClassifier(nn.Module):
    def __init__(self, num_classes, dropout=0.1):
        super(BERTClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1", trust_remote_code=True, use_safetensors=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.fc(self.dropout(pooled_output))

# === Train / Eval Functions ===
def train_epoch(model, loader, loss_fn, optimizer, scheduler, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for batch in tqdm(loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return total_loss / total, correct / total

def eval_epoch(model, loader, loss_fn, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / total, correct / total


# === Grid Search Setup ===
param_grid = {
    "learning_rate": [3e-5, 1e-5, 2e-5],
    "batch_size": [16, 32],
    "dropout": [0.15, 0.1, 0.2],
    "weight_decay": [0.01, 0.05],
    "warmup_proportion": [0.05, 0.1, 0.15],
    "boost_factor": [1.0, 1.3, 1.5, 2.0]
}

all_params = list(ParameterGrid(param_grid))

csv_file = "gridsearch_results.csv"
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(list(param_grid.keys()) + ["val_accuracy"])

best_overall_acc = 0
best_overall_params = None

for params in all_params:
    print(f"\n🚀 Trying config: {params}")

    # Clear GPU
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    # Dataset
    train_dataset = IntentDataset(train_df["text"], train_df["label"], tokenizer)
    val_dataset = IntentDataset(val_df["text"], val_df["label"], tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params["batch_size"])

    # Class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(train_df['label']), y=train_df['label'])
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

    # Boost underperforming classes
    boost_labels = ["treatment", "treatment method", "symptoms", "disease manifestations"]
    boost_indices = label_encoder.transform(boost_labels)
    for idx in boost_indices:
        class_weights_tensor[idx] *= params["boost_factor"]

    model = BERTClassifier(num_classes, dropout=params["dropout"]).to(device)
    loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)
    optimizer = AdamW(model.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])

    num_epochs = 10
    total_steps = len(train_loader) * num_epochs
    warmup_steps = int(params["warmup_proportion"] * total_steps)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

    best_val_acc = 0
    patience = 2
    patience_counter = 0

    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(model, train_loader, loss_fn, optimizer, scheduler, device)
        val_loss, val_acc = eval_epoch(model, val_loader, loss_fn, device)
        print(f"Epoch {epoch+1}: train_acc={train_acc:.4f}, val_acc={val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("⛔ Early stopping")
                break

    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([params[k] for k in param_grid.keys()] + [round(best_val_acc, 4)])

    if best_val_acc > best_overall_acc:
        best_overall_acc = best_val_acc
        best_overall_params = params

print("\n✅ Grid search complete. Results saved to:", csv_file)
print(f"\n🎉 Best overall val_accuracy: {best_overall_acc:.4f}")
print("Best hyperparameters:")
for k, v in best_overall_params.items():
    print(f"  {k}: {v}")


c:\Users\user\Desktop\Souharda\myenv\lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



🚀 Trying config: {'batch_size': 16, 'boost_factor': 1.0, 'dropout': 0.15, 'learning_rate': 3e-05, 'warmup_proportion': 0.05, 'weight_decay': 0.01}


Evaluating: 100%|██████████| 95/95 [00:09<00:00, 10.04it/s]


Epoch 1: train_acc=0.3888, val_acc=0.5236


Evaluating: 100%|██████████| 95/95 [00:09<00:00, 10.02it/s]


...

Epoch 10: train_acc=0.9873, val_acc=0.8009

🚀 Trying config: {'batch_size': 16, 'boost_factor': 1.0, 'dropout': 0.15, 'learning_rate': 3e-05, 'warmup_proportion': 0.15, 'weight_decay': 0.01}


Evaluating: 100%|██████████| 95/95 [00:09<00:00,  9.70it/s]


Epoch 1: train_acc=0.3096, val_acc=0.4891


Training:  88%|████████▊ | 662/754 [03:25<00:28,  3.25it/s]

: 